In [1]:
from mycode.common import *
SEPERATOR = "_TABLE_"

构造数据

In [2]:
from pandas import DataFrame

metadata, tables = fetch_data_from_sqlite(path='./mycode/data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
result_table = None
x_table = ["BookLoan", "Book", "Library", "Student", "Enrollment", "Submission"]
x_key = [None, 'book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
x_how.insert(0, None)
result_table_columns_map = {}
# print(f"{len(result_table)=}")
original_tables = {
    key: tables[key] for key in x_table
}

In [3]:
set(tables["Book"].columns)

{'author', 'book_id', 'isbn', 'library_id', 'title'}

In [4]:
keys = set(x_key) - {None}
for i, x_table_name in enumerate(x_table):
    columns = set(tables[x_table_name].columns)
    result_table_columns_map[x_table_name] = {}
    unique_columns = columns - keys # 除去ref key
    reference_columns = keys.intersection(columns)
    temp = tables[x_table_name]
    if unique_columns:
        convert_columns = {x: x_table_name + SEPERATOR + x for x in unique_columns}
        temp = temp.rename(columns=convert_columns)
        result_table_columns_map[x_table_name].update({
            value: key for key, value in convert_columns.items()
        })
    if reference_columns:
        convert_columns = {x: "REF" + SEPERATOR + x for x in reference_columns}
        temp = temp.rename(columns=convert_columns)
        result_table_columns_map[x_table_name].update({
            value: key for key, value in convert_columns.items()
        })
    # x_table_name
    if result_table is not None:
        on_key = "REF" + SEPERATOR + x_key[i]
        result_table = pd.merge(result_table, temp, on=on_key, how=x_how[i])
    else:
        result_table = temp
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
#result_table

10000 10000 10000 10000 9999 10798 23


In [5]:
# result_table = result_table.sample(n=1000)
result_table

,BookLoan_TABLE_loan_id,REF_TABLE_book_id,REF_TABLE_student_id,BookLoan_TABLE_loan_date,BookLoan_TABLE_return_date,Book_TABLE_title,Book_TABLE_author,Book_TABLE_isbn,REF_TABLE_library_id,Library_TABLE_name,...,Student_TABLE_date_of_birth,Student_TABLE_major,Enrollment_TABLE_enrollment_id,Enrollment_TABLE_course_id,Enrollment_TABLE_semester,Enrollment_TABLE_grade,Submission_TABLE_submission_id,Submission_TABLE_assignment_id,Submission_TABLE_submission_date,Submission_TABLE_grade
0,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,2462,9937,2022-10-18 02:02:31,B
1,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,3984,4403,2022-11-15 20:13:23,I
2,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,7851,6252,2022-07-31 08:44:12,A
3,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,2462,9937,2022-10-18 02:02:31,B
4,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,3984,4403,2022-11-15 20:13:23,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10793,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3450,6477,Spring 2021,F,8054,1227,2022-09-20 01:25:41,C
10794,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3757,4480,Fall 2019,None,578,6739,2022-11-01 16:39:13,A
10795,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3757,4480,Fall 2019,None,8054,1227,2022-09-20 01:25:41,C
10796,9999,9999,2281,2023-12-28,2025-08-23,Challenge last suffer.,Robert Holland,978-1-67998-842-4,9999,Williams-Smith Library,...,1996-11-30,1005,9064,7483,Fall 2021,B,9247,1546,2022-08-05 15:48:08,B


In [6]:
result_table_columns_map

{'BookLoan': {'BookLoan_TABLE_return_date': 'return_date',
  'BookLoan_TABLE_loan_id': 'loan_id',
  'BookLoan_TABLE_loan_date': 'loan_date',
  'REF_TABLE_student_id': 'student_id',
  'REF_TABLE_book_id': 'book_id'},
 'Book': {'Book_TABLE_title': 'title',
  'Book_TABLE_author': 'author',
  'Book_TABLE_isbn': 'isbn',
  'REF_TABLE_library_id': 'library_id',
  'REF_TABLE_book_id': 'book_id'},
 'Library': {'Library_TABLE_name': 'name',
  'Library_TABLE_location': 'location',
  'REF_TABLE_library_id': 'library_id'},
 'Student': {'Student_TABLE_name': 'name',
  'Student_TABLE_email': 'email',
  'Student_TABLE_date_of_birth': 'date_of_birth',
  'Student_TABLE_major': 'major',
  'REF_TABLE_student_id': 'student_id'},
 'Enrollment': {'Enrollment_TABLE_course_id': 'course_id',
  'Enrollment_TABLE_enrollment_id': 'enrollment_id',
  'Enrollment_TABLE_grade': 'grade',
  'Enrollment_TABLE_semester': 'semester',
  'REF_TABLE_student_id': 'student_id'},
 'Submission': {'Submission_TABLE_submission_id':

检查metadata

In [7]:
result_table.to_csv("test_10k_label.csv", index=False)
dataset_csv = "test_10k_label.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-10-16 00:38:53.747 | INFO     | sdgx.data_models.metadata:from_dataloader:294 - Inspecting metadata...
2024-10-16 00:38:53.764 | DEBUG    | sdgx.manager:register:99 - Register for new model: subsetrelationshipinspector
2024-10-16 00:38:53.765 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailinspector
2024-10-16 00:38:53.766 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandidinspector
2024-10-16 00:38:53.766 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandmobilephoneinspector
2024-10-16 00:38:53.767 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandpostcode
2024-10-16 00:38:53.767 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandunifiedsocialcreditcode
2024-10-16 00:38:53.768 | DEBUG    | sdgx.manager:register:99 - Register for new model: chinamainlandaddressinspector
2024-10-16 00:38:53.768 | DEBUG    | sdgx.manager:register:99 - Register for new mode

{'BookLoan_TABLE_loan_date',
 'BookLoan_TABLE_return_date',
 'Book_TABLE_author',
 'Book_TABLE_isbn',
 'Book_TABLE_title',
 'Enrollment_TABLE_grade',
 'Enrollment_TABLE_semester',
 'Library_TABLE_location',
 'Library_TABLE_name',
 'Student_TABLE_date_of_birth',
 'Student_TABLE_email',
 'Student_TABLE_name',
 'Submission_TABLE_grade',
 'Submission_TABLE_submission_date'}

In [8]:
loan_metadata.datetime_columns

{'BookLoan_TABLE_loan_date',
 'BookLoan_TABLE_return_date',
 'Student_TABLE_date_of_birth',
 'Submission_TABLE_submission_date'}

In [9]:
loan_metadata.column_list.__len__()

23

In [10]:
escape_key = "Submission" + SEPERATOR + "submission_date" 

loan_metadata.datetime_format = {
    key: "%Y-%m-%d" for key in loan_metadata.datetime_columns if key != escape_key
}
loan_metadata.datetime_format[escape_key] = "%Y-%m-%d %H:%M:%S"
loan_metadata.datetime_format

{'BookLoan_TABLE_return_date': '%Y-%m-%d',
 'BookLoan_TABLE_loan_date': '%Y-%m-%d',
 'Student_TABLE_date_of_birth': '%Y-%m-%d',
 'Submission_TABLE_submission_date': '%Y-%m-%d %H:%M:%S'}

In [11]:
loan_metadata.discrete_columns = set([
    key for key in loan_metadata.discrete_columns if key not in loan_metadata.datetime_columns
])
# loan_metadata.label_columns = loan_metadata.discrete_columns
#loan_metadata.label_columns.add("grade_x")
#loan_metadata.label_columns.add("grade_y")
print(f"{loan_metadata.int_columns=}\n{loan_metadata.float_columns=}\n{loan_metadata.const_columns=}\n{loan_metadata.bool_columns=}\n{loan_metadata.discrete_columns=}")

loan_metadata.int_columns={'BookLoan_TABLE_loan_id', 'Enrollment_TABLE_course_id', 'Enrollment_TABLE_enrollment_id', 'Student_TABLE_major', 'REF_TABLE_student_id', 'REF_TABLE_book_id', 'Submission_TABLE_assignment_id', 'Submission_TABLE_submission_id', 'REF_TABLE_library_id'}
loan_metadata.float_columns=set()
loan_metadata.const_columns=set()
loan_metadata.bool_columns=set()
loan_metadata.discrete_columns={'Library_TABLE_name', 'Submission_TABLE_grade', 'Book_TABLE_author', 'Enrollment_TABLE_semester', 'Enrollment_TABLE_grade', 'Library_TABLE_location', 'Book_TABLE_title', 'Student_TABLE_email', 'Book_TABLE_isbn', 'Student_TABLE_name'}


In [12]:
loan_metadata.column_encoder = {
    key: "label" for key in loan_metadata.discrete_columns
}

训练

In [13]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    metadata=loan_metadata,
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()

# synthesizer.save("test_model_100k")

2024-10-16 00:38:57.838 | DEBUG    | sdgx.manager:register:99 - Register for new model: outliertransformer
2024-10-16 00:38:57.839 | DEBUG    | sdgx.manager:register:99 - Register for new model: numericvaluetransformer
2024-10-16 00:38:57.840 | DEBUG    | sdgx.manager:register:99 - Register for new model: nonvaluetransformer
2024-10-16 00:38:57.840 | DEBUG    | sdgx.manager:register:99 - Register for new model: emptytransformer
2024-10-16 00:38:57.841 | DEBUG    | sdgx.manager:register:99 - Register for new model: constvaluetransformer
2024-10-16 00:38:57.841 | DEBUG    | sdgx.manager:register:99 - Register for new model: columnordertransformer
2024-10-16 00:38:57.842 | DEBUG    | sdgx.manager:register:99 - Register for new model: emailgenerator
2024-10-16 00:38:57.843 | DEBUG    | sdgx.manager:register:99 - Register for new model: chnpiigenerator
2024-10-16 00:38:57.843 | DEBUG    | sdgx.manager:register:99 - Register for new model: intvalueformatter
2024-10-16 00:38:57.844 | DEBUG   

Preparing data:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-16 00:38:58.924 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:133 - Fitting continuous column BookLoan_TABLE_loan_id...
2024-10-16 00:38:59.652 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:133 - Fitting continuous column REF_TABLE_book_id...
2024-10-16 00:39:00.198 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:133 - Fitting continuous column REF_TABLE_student_id...
2024-10-16 00:39:00.723 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:133 - Fitting continuous column BookLoan_TABLE_loan_date...
2024-10-16 00:39:01.291 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:133 - Fitting continuous column BookLoan_TABLE_return_date...
2024-10-16 00:39:01.828 | DEBUG    | sdgx.models.components.optimize.sdv_ctgan.data_transformer:fit:128 - Fitting discrete column Book_TABLE_title...
2024-10-16 00:39:01.849 | DEBUG    | sdgx.models.components.o

Transforming data:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-16 00:39:13.522 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:253 - Sampling data.
2024-10-16 00:39:13.543 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:258 - Initialize Generator.
2024-10-16 00:39:13.546 | INFO     | sdgx.models.ml.single_table.ctgan:fit:223 - CTGAN prefit finished, start CTGAN training.
2024-10-16 00:39:13.548 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:276 - Fit using data_size:10798, data_dim: 181.


[ColumnTransformInfo(column_name='BookLoan_TABLE_loan_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=10, activation_fn='softmax')], output_dimensions=11), ColumnTransformInfo(column_name='REF_TABLE_book_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=10, activation_fn='softmax')], output_dimensions=11), ColumnTransformInfo(column_name='REF_TABLE_student_id', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=10, activation_fn='softmax')], output_dimensions=11), ColumnTransformInfo(column_name='BookLoan_TABLE_loan_date', column_type='continuous', transform=ClusterBasedNormalizer(model_missing_values=True), output_info=[SpanInfo(dim=1, activation_fn='tanh'), SpanInfo(dim=10

2024-10-16 00:39:14.159 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:302 - Starting model training, epochs: 1


Fitting batches:   0%|          | 0/21 [00:00<?, ?it/s]

2024-10-16 00:39:15.806 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:383 - Epoch 1, Loss G:  2.0333, Loss D:  0.0552, Time:  1.6454
2024-10-16 00:39:15.807 | INFO     | sdgx.models.ml.single_table.ctgan:fit:225 - CTGAN training finished.
2024-10-16 00:39:15.808 | INFO     | sdgx.synthesizer:fit:333 - Model fit... Finished


In [14]:
data_res : DataFrame = synthesizer.sample(1000)

2024-10-16 00:39:15.815 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

Sampling batches:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Inverse transforming:   0%|          | 0/22 [00:00<?, ?it/s]

2024-10-16 00:39:19.884 | INFO     | sdgx.data_processors.transformers.nan:reverse_convert:155 - Data reverse-converted by NonValueTransformer.
2024-10-16 00:39:19.885 | INFO     | sdgx.data_processors.transformers.outlier:reverse_convert:120 - Data reverse-converted by OutlierTransformer (No Action).
2024-10-16 00:39:20.096 | INFO     | sdgx.data_processors.formatters.int:reverse_convert:64 - Data reverse-converted by IntValueFormatter.
2024-10-16 00:39:20.096 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:165 - Data reverse-converting by DatetimeFormatter...
2024-10-16 00:39:20.097 | INFO     | sdgx.data_processors.formatters.datetime:reverse_convert:167 - parameters : ['BookLoan_TABLE_return_date', 'BookLoan_TABLE_loan_date', 'Student_TABLE_date_of_birth', 'Submission_TABLE_submission_date'], {'BookLoan_TABLE_return_date': '%Y-%m-%d', 'BookLoan_TABLE_loan_date': '%Y-%m-%d', 'Student_TABLE_date_of_birth': '%Y-%m-%d', 'Submission_TABLE_submission_date': '%Y-%m-%

In [15]:
data_res.to_csv("test_10k_simulate.csv", index=False)

In [16]:
data_res

,BookLoan_TABLE_loan_id,REF_TABLE_book_id,REF_TABLE_student_id,BookLoan_TABLE_loan_date,BookLoan_TABLE_return_date,Book_TABLE_title,Book_TABLE_author,Book_TABLE_isbn,REF_TABLE_library_id,Library_TABLE_name,...,Student_TABLE_date_of_birth,Student_TABLE_major,Enrollment_TABLE_enrollment_id,Enrollment_TABLE_course_id,Enrollment_TABLE_semester,Enrollment_TABLE_grade,Submission_TABLE_submission_id,Submission_TABLE_assignment_id,Submission_TABLE_submission_date,Submission_TABLE_grade
0,6783,8232,5174,2024-09-03,2024-10-13,Officer what walk.,Mary Marshall,978-1-06-336464-3,4051,Garner-Austin Library,...,2002-08-17,10069,1510,7747,Winter 2020,B,7103,1839,2022-05-30 12:32:51,F
1,2320,3820,2225,2024-05-25,2024-12-12,Lay ready make.,Marissa Lopez,978-1-4706-9658-0,8622,"Rivera, Sullivan and Torres Library",...,1992-09-06,9418,2378,7184,Winter 2023,F,9033,4055,2022-03-10 13:14:38,B
2,1751,3245,8182,2024-07-31,2025-02-18,Model month.,John Brady,978-1-80036-388-5,8497,Scott-Nelson Library,...,1993-05-07,7625,-4,10626,Spring 2021,A,9549,3567,2022-06-28 14:06:57,C
3,6598,204,8663,2024-07-19,2024-11-29,Learn law instead.,Zoe Davis,978-1-4380-2564-3,4006,"Duffy, Phillips and Davis Library",...,1994-10-17,6650,4070,1715,Spring 2020,D,8309,2820,2022-01-14 02:33:46,C
4,2640,5973,5607,2024-06-13,2025-08-10,Thousand control see.,Zoe Davis,978-1-58603-147-3,831,"Levine, Ford and Navarro Library",...,1998-06-09,2107,1146,1904,Fall 2023,,9647,4608,2022-03-20 16:49:14,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9310,6791,3349,2024-06-04,2025-01-03,Play language your.,Lisa Stewart,978-1-4768-6309-2,832,"Robinson, Cohen and Nixon Library",...,1993-05-22,934,8120,1480,Fall 2019,,4392,5221,2022-12-15 08:30:45,B
996,872,253,5493,2024-07-14,2025-05-16,Real property.,Kim Hawkins,978-1-03-822184-1,7196,"Roberts, House and Smith Library",...,2000-07-22,1884,7660,5567,Winter 2019,D,5510,2412,2022-05-28 11:27:31,D
997,1599,1105,4184,2024-09-10,2025-01-09,Throw end.,Lisa Sanchez,978-0-632-18101-8,362,"Lane, Young and Jacobs Library",...,1999-09-18,6908,8305,5217,Spring 2019,F,10428,8264,2022-07-16 21:24:57,F
998,4415,5254,4411,2023-10-19,2024-09-22,Price almost.,Kaitlyn Clark,978-1-229-56867-0,2046,Mcdowell-Brooks Library,...,2004-06-28,1662,3967,8944,Winter 2023,,7728,981,2022-02-14 15:12:35,A


In [17]:
synthesized_tables = {
    table_name: data_res[table_map.keys()].rename(columns=table_map) for table_name, table_map in result_table_columns_map.items()
}

In [18]:
synthesized_tables["Book"]

,title,author,isbn,library_id,book_id
0,Officer what walk.,Mary Marshall,978-1-06-336464-3,4051,8232
1,Lay ready make.,Marissa Lopez,978-1-4706-9658-0,8622,3820
2,Model month.,John Brady,978-1-80036-388-5,8497,3245
3,Learn law instead.,Zoe Davis,978-1-4380-2564-3,4006,204
4,Thousand control see.,Zoe Davis,978-1-58603-147-3,831,5973
...,...,...,...,...,...
995,Play language your.,Lisa Stewart,978-1-4768-6309-2,832,6791
996,Real property.,Kim Hawkins,978-1-03-822184-1,7196,253
997,Throw end.,Lisa Sanchez,978-0-632-18101-8,362,1105
998,Price almost.,Kaitlyn Clark,978-1-229-56867-0,2046,5254


In [19]:
def save_xlsx(tables, name):
    # 使用ExcelWriter保存为多sheet的Excel文件
    with pd.ExcelWriter(name) as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [20]:
def read_xlsx(name):
    # 读取多sheet的Excel文件
    sheets_dict = pd.read_excel(name, sheet_name=None)
    return sheets_dict

In [21]:
save_xlsx(synthesized_tables, "test_10k_simulate_split.xlsx")
save_xlsx(original_tables, "test_10k_label_split.xlsx")